Dataset Features:
1. id: unique ID for a News Article
2. title: the title of a News Article
3. author: author of the News Article
4. text: the text of the article, could be incomplete
5. label: a label that denotes whether the article is fake of real

1 : Fake News

0 : Real News


Import the Libraries

In [43]:
import numpy as np
import pandas as pd
import re # will be used to remove regular expressions in a text
from nltk.corpus import stopwords # corpus is the content of the text, nltk = natural language toolkit
from nltk.stem.porter import PorterStemmer # stemming takes a word and removes prefix and suffix and return root word
from sklearn.feature_extraction.text import TfidfVectorizer # used to convert text data into numerical data 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Downloading the stopwords

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Printing the stopwords in English Language

In [5]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Preprocessing

In [6]:
# Load Dataset into a pandas DataFrame
news_dataset = pd.read_csv('/content/train.csv')

In [7]:
news_dataset.shape

(20800, 5)

In [8]:
# Print the first 5 Rows of our dataset
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
# Counting the Number of Missing values in the dataset
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [11]:
# Replacing the missing values with empty strings
news_dataset = news_dataset.fillna('')

In [12]:
# Merging the Author Name and News Title
news_dataset['content'] = news_dataset['author'] + ' ' +news_dataset['title'] 

In [14]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


Stemming 

Stemming is the process of reducing a word to its root word by removing its prefixes and suffixes

example:-
actor, actress, acting -> root word = act

In [17]:
port_stem = PorterStemmer()

In [18]:
def stemming(content):
  # remove all number and punctuations from the content and replace by ' ' blank space
  stemmed_content =  re.sub('[^a-zA-Z]',' ', content)
  # convert everything to lower case
  stemmed_content = stemmed_content.lower()
  # convert everything in the content into a list
  stemmed_content = stemmed_content.split()
  # stemming each word in list if it isn't a stopword
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  # we are joining all the words back into a string from the list
  stemmed_content = ' '.join(stemmed_content)
  # return stemmed string
  return stemmed_content

In [19]:
# apply takes a function and applies it to all values of pandas series.
news_dataset['content'] = news_dataset['content'].apply(stemming)

Stemmed content

In [20]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [25]:
# Seperating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [24]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [26]:
print(Y)
#  1-> fake
#  0-> Real

[1 0 1 ... 0 1 1]


In [27]:
Y.shape

(20800,)

Converting the textual data to Numerical Data

In [28]:
# tf = term frequency idf = inverse document frequency
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [ ]:
print(X)

Splitting Dataset into training and test data

In [33]:
# here stratify=Y is used to distribute the data equally among real and fake ones into both train and the test data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

Training the Logistic Regression Model

In [34]:
model = LogisticRegression()

In [35]:
model.fit(X_train, Y_train)

LogisticRegression()

Model Evaluation using Accuracy Score

In [39]:
# Accuracy Score for training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)
print("Accuracy Score for training data =",training_data_accuracy)

Accuracy Score for training data = 0.9865985576923076


In [40]:
# Accuracy Score for testing data
X_test_prediction = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_prediction, Y_test)
print("Accuracy Score for testing data =",testing_data_accuracy)

Accuracy Score for testing data = 0.9790865384615385


Making a Predictive System for our model

In [41]:
X_new = X_test[0]
prediction = model.predict(X_new)
print(prediction)

if(prediction[0] == 0):
  print("The News is Real")
else:
  print("The News is Fake")

[1]
The News is Fake


In [42]:
print(Y_test[0])

1
